# Constrained Causal Bayesian Optimization

## Introduction

This notebook contains code for running the 'Constrained Causal Bayesian Optimization' method introduced in the ICML 2023 paper ["Constrained Causal Bayesian Optimization"](https://arxiv.org/abs/2305.20011) by Virginia Aglietti, Alan Malek, Ira Ktena and Silvia Chiappa.

### Abstract

We propose constrained causal Bayesian optimization (cCBO), an approach for finding interventions in a known causal graph that optimize a target variable *under some constraints*. cCBO first reduces the search space by exploiting the graph structure and, if available, an observational dataset; and then solves the restricted optimization problem by modelling target and constraint quantities using Gaussian processes and by sequentially selecting interventions via a constrained expected improvement acquisition function. We propose different surrogate models that enable to integrate observational and interventional data while capturing correlation among effects with increasing levels of sophistication. We evaluate cCBO on artificial and real-world causal graphs showing successful trade off between fast convergence and percentage
of feasible interventions.

## Implementation details

Note that the code requires `python3.10` and `python3.10-dev`. Installation instructions are given in the `README.md` file.

## Notebook overview:
* We first define the list of algorithms we wish to run on a specific SCM example and set their parameters. This Colab can be used to run cCBO with different surrogate models as well as the baseline methods (random and CBO).
* We then select the SCM example we wish to run the experiment for and compute their associated constrained and unconstrained optimum. A full list
of available SCM examples is given in `experiments/data.py`.
* We generate observational and interventional data from the selected example that are then used to initialize the algorithms.
* We finally run the selected algorithms for a given number of trials and plot their convergence to the global optimum (constrained or unconstrained).

In [ ]:
# Set working directory to the parent directory of ccbo
import os
os.chdir('..')

In [ ]:
#@title Importing Packages
from ccbo.methods import ccbo
from ccbo.methods import cbo
from ccbo.methods import random
from ccbo.experiments import data
from ccbo.utils import sampling_utils
from ccbo.utils import initialisation_utils
from ccbo.utils import scm_utils
from ccbo.utils import plotting_utils

In [ ]:
#@title Set list of algorithms to run
alg_to_run = ['cbo', 'random', 'ccbo_single_task', 'ccbo_single_task_causal_prior', 'ccbo_multi_task', 'ccbo_multi_task_causal_prior', 'ccbo_dag_multi_task']

In [ ]:
#@title Set alg parameters

# Number of trials to run the alg for
# N.B. Decrese this value to a small number e.g. 3 to quickly run the algs
n_trials = 50
# Grid to use to optimize acquisition function
n_grid_points = 100
# Optimize on a grid of point or on sampled points
sample_anchor_points = False
# Number of sample to use to generate the ground truth target effect
n_samples_ground_truth = 100
# Sampling seed for the ground truth and the sampling of the target function
sampling_seed = 1
# Whether to use noisy observations of the target and the constraints
noisy_observations = True
# Whether to produce plot and print statistics
verbose = False
# Number of restarts of GP optimization
n_restart = 1
# Number of observations
n_samples_obs = 100
# Number of samples for each intervention
n_samples_per_intervention = 100
# Number of samples for kernel eval
n_kernel_samples = 10
# Use hyper-prior for GP hyper-parameters
hp_prior = True
# Are data points noisy
noisy_observations = True

In [ ]:
#@title Import SCM example, set MIS perturbations and interventional domains
example = data.EXAMPLES_DICT["synthetic1"]()
scm = example.structural_causal_model(
    variables=("X", "Z"), lambdas=(1., 2.))
constraints = scm.constraints
graph = scm.graph
exploration_sets = (("X",), ("Z",))
intervention_domain = {"X": [-3, 2], "Z": [-1, 1]}


In [ ]:
#@title Initialize SCM (un)constrained optimum
(_, _, intervention_domain, all_ce,
  _, constraints_values, optimal_unconstrained_y,
  optimal_constrained_y, optimal_constrained_level,
  optimal_constrained_set) = scm.setup(
      n_grid_points=n_grid_points,
      exploration_sets=list(exploration_sets),
      n_samples=n_samples_ground_truth,
      sampling_seed=sampling_seed)


In [ ]:
#@title Generate observational data
d_o = sampling_utils.sample_scm(
    scm_funcs=scm.scm_funcs,
    graph=None,
    n_samples=n_samples_obs,
    compute_moments=False,
    seed=sampling_seed)

In [ ]:
#@title Generate interventional data
d_i = {k: None for k in exploration_sets}

for var, level in zip(exploration_sets, ((1.,), (0.,))):
  initialisation_utils.assign_interventions(
      variables=var,
      levels=level,
      n_samples_per_intervention=n_samples_per_intervention,
      sampling_seed=sampling_seed,
      d_i=d_i,
      graph=graph,
      scm_funcs=scm.scm_funcs)

In [ ]:
#@title Run algorithms
results = {}
for alg_name in alg_to_run:
  use_causal_prior = alg_name in ["ccbo_single_task_causal_prior", "ccbo_dag_multi_task"]
  is_multi_task = alg_name in ["ccbo_multi_task", "ccbo_multi_task_causal_prior", "ccbo_dag_multi_task"]
  use_prior_mean = alg_name in ["ccbo_single_task_causal_prior", "ccbo_multi_task_causal_prior", "ccbo_dag_multi_task"]

  # Set input params for the algorithm
  input_params = {
      "graph": graph,
      "scm": scm,
      "make_scm_estimator": scm_utils.build_fitted_scm,
      "exploration_sets": list(exploration_sets),
      "observation_samples": d_o,
      "intervention_samples": d_i,
      'number_of_trials': n_trials,
      "intervention_domain": intervention_domain,
      "sample_anchor_points": sample_anchor_points,
      "num_anchor_points": n_grid_points,
      "sampling_seed": sampling_seed,
      "n_restart": n_restart,
      "causal_prior": use_causal_prior,
      "hp_prior": hp_prior,

      # Noisy observations
      "noisy_observations": noisy_observations,
      "n_samples_per_intervention": n_samples_per_intervention
  }

  if alg_name == "cbo":
    alg = cbo.CBO(**input_params)
  elif alg_name == "random":
    alg = random.Random(**input_params)
  else:
    # Add constraints
    input_params["ground_truth_constraints"] = constraints_values
    input_params["constraints"] = constraints
    input_params["multi_task_model"] = is_multi_task
    input_params["use_prior_mean"] = use_prior_mean

    if alg_name == "ccbo_dag_multi_task":
      # Monte Carlo construction of the kernel
      input_params["n_kernel_samples"] = n_kernel_samples

    alg = ccbo.CCBO(**input_params)


  # Run
  alg.run()


  results[alg_name] = alg.optimal_outcome_values_during_trials

In [ ]:
#@title Plot convergence
plotting_utils.plot_outcome(
      n_trials,
      outcomes=list(results.values()),
      labels=list(results.keys()),
      title = "CBO, Random and cCBO with different surrogate models",
      true_objective_values=optimal_constrained_y)
